In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
from scipy.stats import chi2

In [2]:
# define reward function
def get_noise():
    return  np.random.normal(0,1,1)[0]


def get_unitv(d):
    # generate unit vector in sphere
    x = np.random.normal(0,1,d)
    return x/np.linalg.norm(x)

def get_Actions(num,d):
    return np.array([get_unitv(d) for i in range(num)])
    

# stochastic regression
def ECB(n, Actions):
    d = Actions.shape[1]
    initial_run = Actions.shape[0]
    theta = np.array([0.3]*d).reshape(-1,1)
    
    # data matrix
    X = Actions.copy()
    
    # contextual bandits - action set
    y = X@theta + np.array([get_noise() for i in range(initial_run)]).reshape(-1,1)
    
    y = y.reshape(-1)
    nums = np.ones(initial_run)
    rewards = y.copy().reshape(-1)
    
    for i in range(n-initial_run):
        arm_means = rewards/nums
        arm = np.argmax(arm_means)
        # defining decaying rate
        flag = np.random.uniform(0,1,1)[0]
        if flag > np.log(i+1)**2/(i+1) and i > 50:
            arm = np.random.randint(1,1+ initial_run) - 1
        
        # append new observation
        newx = Actions[arm,:]
        X = np.append(X, newx.reshape(1,-1), axis = 0 )
        newr = np.sum(newx*0.3)  + get_noise()
        y = np.append(y, newr)
        
    return X, y

## $\Sigma_0 = \log(n)$

In [3]:
def Volumn(V):
    # given a matrix V, compute the volumn correspoing to the ellipse of the format x^T V x \leq 1
    # universal constant are dropped
    s =  np.linalg.svd(V)[1]
    return 1/np.product(np.sqrt(s))



def scaled_estimators(n, Actions, wlam, clam, sampler, alpha_list): # need to add lambda as tuning parameter here. 
    
    d = Actions.shape[1]
    # consider confidence level 
    siglevel = 1 - alpha_list 
    siglevelLen = np.size(siglevel)
    cql = chi2.ppf(siglevel, d)
    
    true_theta = np.array([0.3]*d).reshape(-1,1)
    CI_data = np.zeros((2, siglevelLen * 4 )) # KK -- first row is coverage and second row is volume. Times 4 means 4 methods
    # OLS, W decor, concentration and ALEE (asymptotic) ? 
    
    X,y = sampler(n, Actions)
    #######################################
    # compute OLS
    M = LinearRegression(fit_intercept = False).fit(X, y)
    coeff = M.coef_.copy().reshape(-1,1)
    Sn = X.T@X
    residual = y.reshape(-1,1) - X@coeff
    # compute residual and use it as an estimate of sigma^2
    sig_h = np.sqrt(np.mean(residual**2))
    
    c = 1/sig_h**2*(coeff.reshape(1,-1)- true_theta.reshape(1,-1))@Sn@(coeff - true_theta)
    c = c.reshape(-1)[0]

    ## KK -- what does this part of code do?
    
    for i in range(siglevelLen): # KK --  3 -- is the number of significance level ?
        cq = cql[i]
        cover = c < cq - 0
        v = Volumn(Sn/sig_h**2/cq)
        v = np.log(v)
        CI_data[:,0 + 4*i] = np.array([cover, v]).reshape(-1)  # KK -- coverage values grouped by significance levels 
     
    #######################################
    # W-decorrelation
    W_lambdas = np.ones(n)*wlam
    wols = M.coef_.copy().reshape(-1)
    w = np.zeros((d,1))
    WX = np.zeros((d,d))
    W = np.zeros_like(X)
    for t in range(n):
        w = (np.eye(d) - WX)@X[t,:].reshape(-1,1) 
        w = w/(np.sum(X[t,:]**2) + W_lambdas[t])
        W[t, :] = w.reshape(-1)
        wols += w.reshape(-1) * (y[t] - np.sum(X[t,:]*coeff.reshape(-1)) ) 
        WX = WX + w@X[t,:].reshape(1,-1)
        
    WW = np.linalg.inv(W.T@W)
   
    # compute statistic
    c = 1/sig_h**2*(wols - true_theta.reshape(1,-1))@WW@(wols.reshape(-1,1) - true_theta)
    c = c.reshape(-1)[0]
    for i in range(siglevelLen):    ## KK -- same comment here, is 3 the number of target coverage level? 
        cq = cql[i]
        cover = c < cq - 0
        v = Volumn(WW/sig_h**2/cq)
        v = np.log(v)
        CI_data[:,1 + 4*i] = np.array([cover, v]).reshape(-1)  # coverage grouped by significance levels
    
   
    
    ######################################
    #  Modified ALEE - log(n) # Sigma_0 = log_n hard encoded. 
    Z = np.zeros_like(X)
    Sigma0 = np.log(n)**0.5*np.eye(d)
    for t in range(n):
        if t == 0:
            S = Sigma0
        else:
            S = Sigma0 + X[:t,:].T@X[:t,:]
        
        svd_u, svd_s, svd_v = np.linalg.svd(S, full_matrices=True)
        
        S_invh = svd_u@np.diag(svd_s**(-0.5))@svd_v

        zt = S_invh@X[t,:].reshape(-1,1)
        
        Z[t,:] = zt.reshape(-1)
    
    
    W = np.zeros_like(Z)
    vt = np.eye(d)
    for i in range(n):
        # zi
        z = Z[i,:].reshape(1,-1)
        vtt = vt.copy()
        term_deno = 1 + z@vtt@z.T
        term_deno = term_deno.reshape(-1)[0]
        
        # update
        vt = vtt - vtt@z.T@z@vtt/term_deno
        w = np.sqrt(term_deno)*vt@z.T

        W[i,:] = w.reshape(-1)
        
    
    ######### design new Zt
    kappa = np.log(n)
    tau = np.exp(1)*np.sqrt(np.log(n))
    Z1 = np.copy(Z)
    Vn_inv = np.eye(d) + Z1.T@Z1
    evalues, evectors = np.linalg.eig(Vn_inv)
    
    addn = 0
    # construct new Z
    for i in range(d):
        if evalues[i] < kappa:
            dn = int((kappa - evalues[i])*tau)
            addn = addn + dn
            Z1 = np.concatenate((Z1, np.tile(evectors[:,i], (dn,1))/tau), axis = 0)
    
    
    addW = np.zeros((addn,d))
    
    for i in range(addn):
        z = Z1[i+n,:].reshape(1,-1)
        vtt = vt.copy()
        term_deno = 1 + z@vtt@z.T
        term_deno = term_deno.reshape(-1)[0]
        
        # update
        vt = vtt - vtt@z.T@z@vtt/term_deno
        w = np.sqrt(term_deno)*vt@z.T
        
        addW[i,:] = w.reshape(-1)
    
    # simulate gaussian random variable
    new_error = np.random.normal(0, sig_h, addn).reshape(-1,1)
    
    # design new estimator
    A = W.T@X
    alee = np.linalg.inv(A)@(W.T@y.reshape(-1,1) + addW.T@new_error)
    
    
    OSn = A.T@A

    
    c = 1/sig_h**2*(alee.reshape(1,-1)- true_theta.reshape(1,-1))@OSn@(alee - true_theta)
    c = c.reshape(-1)[0]
    for i in range(siglevelLen):
        cq = cql[i]
        cover = c < cq - 0
        
        v = Volumn(OSn/sig_h**2/cq)
        v = np.log(v)
        CI_data[:,2 + 4*i] = np.array([cover, v]).reshape(-1)  # grouped by significance level.

    
  
    
    ######################################
    # ridge concentration
    # set S
    S = np.sqrt(d)
    rols = np.linalg.inv(X.T@X + clam*np.eye(d))@X.T@y.reshape(-1,1)

    # Adding lambdaPar value - clam
    Vt = clam * np.eye(d) + Sn

    
    delta_list = alpha_list.copy()
    
    c = (rols.reshape(1,-1) - true_theta.reshape(1,-1))@Vt@(rols - true_theta)
    c = c.reshape(-1)[0]
    
    for i in range(siglevelLen):
        delta = delta_list[i]
        ## Needs to be checked 
        cq = sig_h*np.sqrt(np.log(np.linalg.det(Vt)/delta**2) - d*np.log(clam)) + np.sqrt(clam)*S 
        # lambda = 1 and S = |thetastar| \leq 2   
        cover = c < cq**2 - 0
        v = Volumn(Vt/cq**2)
        v = np.log(v)
        CI_data[:,3 + 4*i] = np.array([cover, v]).reshape(-1)
    

    return CI_data
    
    
    
#   replication function
# need to change this part to accomodate siglevels 
# N -- number of replications 
# n -- sample size 

def ACV_repli(N , n, Actions, wlam, clam,  sampler, alpha_list):
    nalpha = np.size(alpha_list)
    
    E = np.zeros((nalpha, N, 8))
    
    for i in range(N):
        print(i)
        output = scaled_estimators(n, Actions, wlam, clam, sampler, alpha_list)
        for k in range(nalpha):
            E[k,i,:] = output[:,(4*k):(4*k+4)].reshape(1,8)
        
    return E

In [4]:
# This is for W-decorrelaton lambda stuff. 

def compute_lam(N, n, sampler, Actions):
    
    record = np.zeros(N)
    for k in range(N):
        
        X,y = sampler(n,Actions)

        evalue, evector = np.linalg.eig(X.T@X)
        
        record[k] = np.min(evalue)
        
    return np.percentile(record, 10)

In [5]:
np.random.seed(517)
Actions = get_Actions(15, 10)
# first input is the number of actions and second one is dimension.

np.random.seed(517)
# compute_lam(N, n, sampler, Actions) 
wlam = compute_lam(1000, 1000, ECB, Actions)/np.log(1000) # this is for W decorrelation 

In [6]:
np.random.seed(517)
Actions = get_Actions(15, 10)

np.random.seed(517)
alpha_list = np.array([0.2, 0.15,0.1])
E = ACV_repli(1000, 1000, Actions, wlam, clam = 0.01, sampler = ECB, alpha_list=  alpha_list )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Confidence level first one

In [7]:
df = pd.DataFrame(np.mean(E[0,:,:], axis = 0).reshape(2,4), columns = ['ols','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,Concentration
0,0.802,0.784,0.926,1.000
1,-7.314,8.373,-1.956,2.509


In [8]:
df = pd.DataFrame(np.std(E[0,:,:], axis = 0).reshape(2,4), columns = ['ols','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,Concentration
0,0.398,0.412,0.262,0.000
1,0.242,0.230,0.239,0.232


In [9]:
df = pd.DataFrame(np.mean(E[1,:,:], axis = 0).reshape(2,4), columns = ['OLS','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,OLS,W,ALEE-logn,Concentration
0,0.859,0.844,0.946,1.00
1,-6.923,8.764,-1.565,2.54


In [10]:
df = pd.DataFrame(np.std(E[1,:,:], axis = 0).reshape(2,4), columns = ['OLS','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,OLS,W,ALEE-logn,Concentration
0,0.348,0.363,0.226,0.000
1,0.242,0.230,0.239,0.232


In [11]:
df = pd.DataFrame(np.mean(E[2,:,:], axis = 0).reshape(2,4), columns = ['ols','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,Concentration
0,0.900,0.901,0.968,1.000
1,-6.447,9.240,-1.089,2.584


In [12]:
df = pd.DataFrame(np.std(E[2,:,:], axis = 0).reshape(2,4), columns = ['OLS','W', 'ALEE-logn', 'Concentration'])
round(df,3)

,OLS,W,ALEE-logn,Concentration
0,0.300,0.299,0.176,0.000
1,0.242,0.230,0.239,0.232
